In [1]:
import pandas as pd
import numpy as np
#데이터 불러오기 
T_data = pd.read_excel('all_real_data.xlsx')
G_data = pd.read_excel('all_garbage_data.xlsx')

T_data = T_data[:30000]

data_df = pd.concat([T_data, G_data])

T_data = None
G_data = None

#행 섞기 
data_df = data_df.sample(frac=1).reset_index(drop=True)
print(data_df.head())


                                                text label
0  해운대룸싸롱 해운대룸바 노래주점 해운대가라오케 그렇게 ★즐 겨용 해운대고구려룸 지금...    유해
1  메타랩스, -4.15% VI 발동메타랩스(090370)가 VI가 발동했다. 전일 대...   부동산
2  베비언스 핑크퐁 유아세제 리필형, 2200ml, 3개향도 좋고 늘 애용하는 제품입니...    진성
3  [아모레 헤어바디 공식몰] 미쟝센 헬로크림 염색약 125g초보라서 꼼꼼하게 바르지 ...    진성
4  정회원 신청합니다************* 신청 양식 ************ ◎ 네이...    광고


In [2]:
#라벨인코더를 사용하여 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data_df['label'] = label_encoder.fit_transform(data_df['label'])

In [3]:
#전처리
##빈 text -> NaN, 중복제거, NaN 행 제거 
data_df['text'] = data_df['text'].str.replace("[^가-힣 ]","")
# data_df['text'] = data_df['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

import numpy as np
data_df['text'].replace('', np.nan, inplace=True)
data_df = data_df.dropna(how='any')
data_df.drop_duplicates(['text'], inplace=True)
data_df.loc[data_df.text.isnull()]


,text,label


In [4]:
from eunjeon import Mecab
mecab = Mecab()

data_df['text'] = data_df['text'].apply(mecab.morphs)
data_df['tokenized'] = data_df['text'].apply(lambda x : [item for item in x if len(item) > 1 ])


KeyboardInterrupt: 

In [ ]:
x = data_df['tokenized']
y = data_df['label']

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [ ]:
print(x_train.head())
print(y_train.head())


In [ ]:
def f(x):
    if x == []:
        return ''
    else:
        return x 

data_df['tokenized'] = data_df['tokenized'].apply(lambda x :f(x))
data_df['tokenized'].replace('', np.nan, inplace=True)
data_df = data_df.dropna(how='any')

In [ ]:
print(len(data_df.index))
# data_df

In [ ]:
#Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(x_train)

In [ ]:
print(tokenizer.word_index)


In [ ]:
threshold = 14
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
vocab_size = len(tokenizer.word_index) -rare_cnt
# vocab_size

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token='OOV')

tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)


In [ ]:
drop_train = [index for index, sentence in enumerate(x_train) if len(sentence) < 1]
drop_train

In [ ]:
x_train = np.delete(x_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

In [ ]:
import matplotlib.pyplot as plt
print('리뷰의 최대 길이 :',max(len(l) for l in x_train))
print('리뷰의 평균 길이 :',sum(map(len, x_train))/len(x_train))
plt.hist([len(s) for s in x_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 600
below_threshold_len(max_len, x_train)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(x_train, maxlen = max_len)
x_test = pad_sequences(x_test, maxlen = max_len)

In [ ]:
category = np.max(y_train)+1
print("카테고리의 종류 : ",category)
print(x_train.shape)
print(y_train.shape)
print(np.unique(y_train))

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Conv1D,Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization,Input, concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

input1 = Input(shape=(None,), dtype='int32')
model1 = Embedding(vocab_size+2, 512)(input1)
model1 = Dropout(0.3)(model1)
model1 = Conv1D(512, 2, padding='valid', activation='relu')(model1)
model1 = GlobalMaxPooling1D()(model1)



# input2 = Input(shape=(None,), dtype='int32')
# model2 = Embedding(vocab_size+2, 512)(input2)
# model2 = Dropout(0.3)(model2)
# model2 = Conv1D(512, 4, padding='valid', activation='relu')(model2)
# model2 =GlobalMaxPooling1D()(model2)



input3 = Input(shape=(None,), dtype='int32')
model3 = Embedding(vocab_size+2, 512)(input3)
model3 = Dropout(0.3)(model3)
model3 = Conv1D(512, 6, padding='valid', activation='relu')(model3)
model3 = GlobalMaxPooling1D()(model3)



concatenate = concatenate([model1, model3])
# concatenate = concatenate([model1, model2, model3])

# concatenate = concatenate([model1])


concatenate = Dense(128, activation='relu')(concatenate)
output = Dense(6, activation='softmax')(concatenate)

model = Model([input1, input2, input3], output)


# model=Sequential()
# model.add(Embedding(vocab_size+2, 512))
# model.add(Dropout(0.3))
# model.add(Conv1D(512, 2, padding='valid', activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(6, activation='softmax'))



In [ ]:
# from tensorflow.keras.layers import Embedding, Dense, LSTM, Conv1D,Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization,Input, concatenate
# from tensorflow.keras.models import Sequential, Model
# from tensorflow.keras.models import load_model
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# input1 = Input(shape=(None,))
# model1 = Embedding(vocab_size+2, 512)(input1)
# model1 = Dropout(0.3)(model1)
# model1 = Conv1D(512, 2, padding='valid', activation='relu')(model1)
# model1 = GlobalMaxPooling1D()(model1)

# output = Dense(6, activation='softmax')(model1)

# model = Model(input1, output)



In [ ]:
model.summary()

In [ ]:
import time
start = time.time()
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.fit([x_train ,x_train], y_train, epochs=100, callbacks=[es], batch_size=60, validation_split=0.2)
# model.fit([x_train, x_train ,x_train], y_train, epochs=100, callbacks=[es], batch_size=60, validation_split=0.2)

# mode = load_model('saved/conv1D_model.h5')
end = time.time()
print(end - start)

In [ ]:
import pickle
model.save('saved/conv1D_ensemble_model.h5')
pickle.dump(tokenizer,open('saved/conv1D_ensemble_tokenizer.pkl','wb'))
pickle.dump(label_encoder,open('saved/conv1D_ensemble_labelencoder.pkl','wb'))



In [ ]:
y_predict = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test.values, y_predict)
print(acc)
print(label_encoder.inverse_transform(y_predict[:10]))